# Pandas Overview — Solutions

This notebook walks through the core pandas patterns that we're learning in this  course. We'll use the car listings dataset throughout so the context stays familiar.

**By the end you'll know how to:**
- Create new columns from existing ones
- Clean string columns efficiently
- Filter rows with `.loc` and `.query()`
- Aggregate data with `groupby` + `agg`

**Part 2** then shows how these pieces combine to answer three questions from Assignment 1 that people found tricky.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# On your computer, just put car_listings.csv in the same folder as this notebook
listings = pd.read_csv('car_listings.csv')

# On my computer, it's more convenient for me *not* to make copies of everything
#listings = pd.read_csv("../../2026/spring/data/car_listings.zip")

# Type conversions (same as every assignment)
listings['time_posted']           = pd.to_datetime(listings['time_posted'], errors='coerce')
listings['year_from_time_posted'] = listings['time_posted'].dt.year
listings['year']       = pd.to_numeric(listings['year'],       errors='coerce').astype('Int64')
listings['odometer']   = pd.to_numeric(listings['odometer'],   errors='coerce').astype('Int64')
listings['post_id']    = pd.to_numeric(listings['post_id'],    errors='coerce').astype('Int64')
listings['num_images'] = pd.to_numeric(listings['num_images'], errors='coerce').astype('Int64')
listings['price']      = pd.to_numeric(listings['price'],      errors='coerce')

print(listings.shape)

---
# Part 1: Building Blocks

## 1. Making New Columns

The basic pattern is `df['new_col'] = <expression>`. The expression runs on the whole column at once — no loop needed.

In [ ]:
# Arithmetic on two columns
listings['car_age'] = listings['year_from_time_posted'] - listings['year']

# Comparison → boolean column  (True/False)
listings['high_mileage'] = listings['odometer'] > 150_000

# Math on two columns — note: produces NaN where either input is missing, inf where odometer == 0
listings['price_per_mile'] = listings['price'] / listings['odometer']

# I _love_ `sample` as a way to look at data in a data frame
listings[['odometer', 'price', 'car_age', 'high_mileage', 'price_per_mile']].sample(10)

### apply with a lambda

When the expression you need is more complex, `apply(lambda x: ...)` runs a custom function element-by-element. The `lambda` is just a compact, anonymous function.

In [ ]:
# Cap odometer at 500,000 to remove obvious data errors, leave NaN as-is
listings['odometer_capped'] = listings['odometer'].apply(
    lambda x: min(x, 500_000) if pd.notna(x) else x
)
# The "x" is the argument that gets passed in. So if you're applying this down a column,
# you'll get each value in the column passed in as "x". 


# Equivalent without apply (faster, but less readable for complex logic):
# listings['odometer_capped'] = listings['odometer'].clip(upper=500_000)

print('Before cap — max odometer:', listings['odometer'].max())
print('After cap  — max odometer:', listings['odometer_capped'].max())

**Try it:** Create a `log_price` column using `np.log(listings['price'])`. What happens to rows where `price` is zero or negative?

In [ ]:
# Your code here

## 2. Cleaning String Columns

String columns in this dataset are inconsistently capitalized (`Ford`, `ford`, `FORD` all exist). We want lowercase everywhere so grouping and comparisons work reliably.

Pandas strings have a `.str` accessor that vectorizes string methods — no loop over rows needed.

In [ ]:
# Naive approach: one line per column — works but gets tedious for 10 columns
# listings['make']  = listings['make'].str.lower()
# listings['model'] = listings['model'].str.lower()
# listings['fuel']  = listings['fuel'].str.lower()
# ...

# Elegant approach: define the list once, loop once
str_cols = ['make', 'model', 'location', 'title', 'fuel',
            'drive', 'transmission', 'paint', 'type', 'condition']

for col in str_cols:
    listings[col] = listings[col].str.lower()

listings[['make', 'model', 'fuel', 'drive']].head(4)

In [ ]:
# While we're at it: drop duplicate post IDs
listings = listings.drop_duplicates(subset='post_id')
print(listings.shape)

## 3. Filtering with `.loc` and `.query()`

Two main tools for selecting rows that meet a condition.

In [ ]:
# .loc[row_condition, columns]  — the row condition is a boolean Series
# Select specific columns at the same time
listings.loc[
    listings['price'] > 40_000,
    ['make', 'model', 'year', 'price', 'odometer']
].head(5)

In [ ]:
# .query() — pass a string expression, often more readable for compound conditions
listings.query('price > 40_000 and make == "ford"').head(5)

# Equivalent .loc:
# listings.loc[(listings['price'] > 40_000) & (listings['make'] == 'ford')]

In [ ]:
# You can reference columns you created — including booleans
listings.query('high_mileage and price < 8_000')[['make', 'model', 'odometer', 'price']].head(5)

**Try it:** In the first assignment, we saw prices of up to \$200,000. Use the cell below to select a few key columns and evaluate the whether or not these prices seem legitimate. 

In [ ]:
# Your code here

**Try it:** Filter to listings in `chicago` with `odometer` under 50,000. How many are there?

In [ ]:
# Your code here

## 4. GroupBy and Aggregation

The split–apply–combine pattern: **split** the data into groups, **apply** a function to each group, **combine** the results into a table.

`.agg()` is the workhorse — it lets you compute multiple statistics at once and give them meaningful names.

In [ ]:
# Single stat: mean price per make, sorted descending
listings.groupby('make')['price'].mean().sort_values(ascending=False).head(8)

In [ ]:
# Multiple stats at once with named aggregation
# Syntax: new_col_name = ('source_col', 'function')
(listings
    .groupby('make',as_index=False)
    .agg(
        n             = ('price', 'count'),
        median_price  = ('price', 'median'),
        mean_price    = ('price', 'mean'))
    .sort_values('mean_price', 
                 ascending=False)
    .head(8)
)

**Try it:** Repeat the code above but add a query statement to it to limit to makes that have at least 50 listings. 

In [ ]:
# Your code here

**Try it:** Find the median `odometer` reading for each `location`. Which location has the highest median mileage?

In [ ]:
# Your code here

**Try it:** Now add the count of listings to the above output. Do you notice think there are any patterns between the number of listings and the median odometer?

In [ ]:
# Your code here

## 5. For Loops vs. Vectorized Operations

If you are coming to pandas from base Python, writing a `for` loop over rows feels natural. It works — but it can be dramatically slower than the pandas equivalents you have been seeing.

Under the hood, pandas delegates column operations to NumPy, which runs compiled C code on entire arrays at once. A Python `for` loop, by contrast, runs through the Python interpreter once per row, which carries significant overhead.

The comparison below uses `iterrows()`, which is the loop-friendly pandas API — and one of the most common traps students fall into.

In [ ]:
import time

sample = listings.sample(100_000, random_state=20260223).copy()

# --- Loop approach using iterrows() ---
start = time.time()
result = []
for _, row in sample.iterrows():
    result.append(row['odometer'] > 150_000)
sample['hm_loop'] = result
loop_time = time.time() - start

# --- Vectorized approach ---
start = time.time()
sample['hm_vec'] = sample['odometer'] > 150_000
vec_time = time.time() - start

print(f'Loop (iterrows):  {loop_time:.3f}s')
print(f'Vectorized:       {vec_time:.6f}s')
print(f'Speedup:          ~{loop_time / vec_time:.0f}x faster')

On 100,000 rows the loop typically takes 1–3 seconds; the vectorized version runs in microseconds — a gap of **1,000x or more**. On the full 250k-row dataset, that difference becomes deeply painful. And remember, the full Carbitrage data set is 3.6 million rows.

**When are loops actually fine?**
- Iterating over a small number of **columns** (not rows) — you saw this in the string-cleaning loop above
- When your per-row logic genuinely cannot be expressed as a column operation (rare, and usually a sign to reach for `apply`)
- When the bottleneck is I/O or an external API call, not computation

The rule of thumb: loops over rows are almost always the wrong tool in pandas. If you find yourself writing `for _, row in df.iterrows()`, pause and ask whether there is a column-level operation that does the same thing.

---
# Part 2: Putting It Together

The three questions below are representative of what students find hardest in Assignment 1. Each one is just a combination of the patterns above.

## Q1: Which make/models have the highest proportion of high-mileage listings?

*(Restrict to make/models with at least 100 listings.)*

**Key insight:** the mean of a boolean column equals the proportion of `True` values.

In [ ]:
(
    listings
    .groupby(['make', 'model'])
    .agg(
        n           = ('high_mileage', 'count'),
        pct_high    = ('high_mileage', 'mean'),   # mean of bool = proportion of True
    )
    .query('n >= 100')
    .sort_values('pct_high', ascending=False)
    .head(10)
)

## Q2: Which makes have the highest median price-per-mile?

*(Restrict to makes with at least 50 listings. Exclude odometer == 0 to avoid division artifacts.)*

In [ ]:
(
    listings
    .query('odometer > 0')          # avoid inf from dividing by zero
    .groupby('make')
    .agg(
        n          = ('price_per_mile', 'count'),
        median_ppm = ('price_per_mile', 'median'),
    )
    .query('n >= 50')
    .sort_values('median_ppm', ascending=False)
    .head(10)
)

## Q3: How many listings were posted each month?

`.dt.to_period('M')` converts a datetime to a Year-Month label (e.g., `2024-09`) that's easy to group on. An alternative would be to do lowercase 'm' (the month number) along with 'y' (the year).

In [ ]:
listings['year_month'] = listings['time_posted'].dt.to_period('M')

by_month = (
    listings
    .groupby('year_month')
    .size()
    .reset_index(name='count')
)
by_month

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))
ax.bar(by_month['year_month'].astype(str), by_month['count'])
ax.set_xlabel('Month')
ax.set_ylabel('Listings')
ax.set_title('Listings Posted per Month')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()